In [1]:
import pandas as pd
import re

##merging data
### This function takes columns and append extra data of unnamed columns inside comments columns
def concat_comments(df):
    comments = []
    for index, row in df.iterrows():
        con_str = row['Comments']
        for x in range(8,len(list(df))): 
            col_name = 'Unnamed: ' + str(x)
            if not pd.isna(row[col_name]):
                con_str = str(con_str)  + ' ' + str(row[col_name])
        comments.append(con_str)
    df['Comments'] = comments    
    df2 = df.iloc[: ,0:8]    
    return df2    

### This function takes columns and append extra columns inside comments columns
def find_inde(patt1,strm):
    try: 
        inde=re.search(patt1,strm).start()
        return inde
    except Exception as e:
        return -1

### This function takes input string (comments) and return the body of all emails inside the  comment
def find_body(inp_str):
    #From:
    li_from=[m.start() for m in re.finditer('From:', inp_str)]
    li_sen=[m.start() for m in re.finditer('Sent:', inp_str)]
    li_to=[m.start() for m in re.finditer('To:', inp_str)]
    from_=[]
    sent_=[]
    To_=[]
    body_=[]
    
    patt1='[Tt]hanks[\s].*and[\s].*[Rr]egards[\s,]'
    patt2='Regards[\s,]'
    patt3='Subject:'
    patt4 = '[Tt]hanks[\s+.]*&[\s+.].*[Rr]egards[\s+,]*'
    patt5 = '[Kk]ind[\s+]*[Rr]egards'
    
#     print(li_from,li_sen,li_to)
    
    if len(li_from)==len(li_sen)==len(li_to):
#         print('dasd')
        for i in range(len(li_from)):
            from_.append(inp_str[li_from[i]:li_sen[i]])
            sent_.append(inp_str[li_sen[i]:li_to[i]])
            if i!=len(li_from)-1:
                end_str=inp_str[li_to[i]:li_from[i+1]]
            else:
                end_str=inp_str[li_to[i]:] 
            if find_inde(patt3,end_str)!=-1:
                ind=find_inde(patt3,end_str) 
                To_.append(end_str[:ind])
                end_str=end_str[ind:]
            if find_inde(patt1,end_str)!=-1:
                ind=find_inde(patt1,end_str)
                end_str=end_str[:ind] 
                
            elif find_inde(patt4,end_str)!=-1:
                ind=find_inde(patt4,end_str)
                end_str=end_str[:ind]
            elif find_inde(patt5,end_str)!=-1:
                ind=find_inde(patt5,end_str)
                end_str=end_str[:ind]
                
            elif find_inde(patt2,end_str)!=-1:
                ind=find_inde(patt2,end_str)
                end_str=end_str[:ind]
            
                
            body_.append(end_str)
    if len(body_)==0:
        return None,None
    return body_[-1],'last email body'    
    
### This function takes input string (comments) and search for string between  ITAUTOEMAILITSDIN and regards 
def itauto_clean(comment):
    sub_str=None
    pat_out=None
    str2 = re.findall(r'ITAUTOEMAILITSDIN[\s+*\w+,.&:()/?=+@!#$%&-“” '';()\_]+', str(comment))
    if not str2:
        sub_str=None
    else:
        sub_str=str2
#         print(sub_str)
        pattern = re.sub(r'\[[\'\"]ITAUTOEMAILITSDIN[*\s+]*',"",str(sub_str))
        pattern = re.sub(r'\'\]',"",pattern)
        pattern = re.sub(r'^[Hh]i[\s+]*[Tt]eam',"",pattern)
        pattern = re.sub(r'^[Hh]i',"",pattern)
        pattern = re.sub(r'^[Hh]ello',"",pattern)
        pattern = re.sub(r'^[Dd]ear[\s+]*it',"",pattern.lower())
        pattern = re.sub(r'^[\s+]*it',"",pattern)
        pattern = re.sub(r'please',"",pattern)
        pattern = re.sub(r'morning',"",pattern)
        sub_str = re.sub(r'^,',"",pattern)
        patt1 = 'thanks'
        patt2 = 'many thanks'
        patt3 = 'regards'
        patt4 = '\*\*this[\s+]*is[\s+]*a[\s+]*peoplesoft[\s+]*generated[\s+]*automatic.\*\*'
        pattern_found = []
        pattern_not =[]
        ####
        str1 = sub_str.lower()
        ind = find_inde(r'thank[s]*',str1)
        if ind != -1:
            sub_str = str1[:ind]
            pat_out = 'itauto'
        else:
            ind = find_inde(patt2,str1)
            if ind != -1:
                sub_str = str1[:ind]
                pat_out = 'itauto'
            else:
                ind = find_inde(patt3,str1)
                if ind != -1:
                    sub_str = str1[:ind]
                    pat_out = 'itauto'
                elif find_inde(patt4,str1) != -1:
                    sub_str = str1[:find_inde(patt4,str1)]
                    pat_out = 'itauto'
                else:
                    sub_str = str1
                    pat_out = 'itautonotregards'
    return sub_str,pat_out

### This function takes input string (comments) and search for string between different pattern     
def find_substr(strm):
    out_str=None
    pattern=None
    if type(strm) == str:
        if 'Issue' in strm  or 'reason' in strm:
            if 'Solution' in strm  or 'Resolution' in strm:
                Flag=False
                in_1=strm.find('Issue')
                in_2=strm.find('reason')
                in_3=strm.find('Solution')
                in_4=strm.find('Resolution')
                if in_1!=-1 and in_3!=-1:
                    out_str=strm[in_1:in_3]
                    pattern='start with Issue end with Solution'
                if in_2!=-1 and in_3!=-1:
                    out_str=strm[in_2:in_3]
                    pattern='start with reason end with Solution'
                if in_1!=-1 and in_4!=-1:
                    out_str=strm[in_1:in_4]
                    pattern='start with Issue end with Resolution'
                if in_2!=-1 and in_4!=-1:
                    out_str=strm[in_2:in_4]
                    pattern='start with reason end with Resolution'
                         
    return  out_str,pattern          
                 
### this function calls find_substr -> itauto_clean -> find_body in sequence to extrac problem from comments  
def find_sol_summary(df3):    
    patt1="summary[\s]*:"
    patt2="solution[\s]*:"
    df_res=pd.DataFrame()
    li_que=[]
    li_pattern=[]
    i=0
    count1=0
    count2=0
    count3=0
    for item in df3['Comments']:
        Flag=True
        Flag2=True
        if type(item) == str:
            strm=item.lower()
            if find_inde(patt1,strm)!=-1:
                start=find_inde(patt1,strm)
                if find_inde(patt2,strm)!=-1:
                    end=find_inde(patt2,strm)
                    li_que.append(strm[start:end])
                    li_pattern.append('start with summary end with solution')
                    Flag=False
        if Flag:
            
            out_str,pattern=find_substr(item)
            out_str1,pattern1=itauto_clean(str(item))
            out_str2,pattern2=find_body(str(item))
            if out_str!=None:
                count1+=1
                li_que.append(out_str)
                li_pattern.append(pattern)
                Flag2=False
                
              
            elif out_str1!=None and Flag2:
                count2+=1
                li_que.append(out_str1)
                li_pattern.append(pattern1)
                Flag2=False
                
#                 
            elif out_str2!=None and Flag2:
                count3+=1
                li_que.append(out_str2)
                li_pattern.append(pattern2)
            else:
                li_que.append(df3['Comments'][i])
                li_pattern.append('')
        i+=1 
    print(count1,count2,count3)  
    
    df3['extracted_problem']= li_que
    df3['pattern_used']= li_pattern


In [4]:
df=pd.read_excel(r"C:\Users\annautiyal\Documents\vQSM Data Extract Jul-19 to Aug'19 (002).xlsx",'Aug-19')
df2=concat_comments(df)
df3=df2.copy()
find_sol_summary(df3)
df3 = df3.reset_index()
# merged_summery_ITAUTO(df3)
df3.to_excel('C:/Users/annautiyal/Documents/vQSM_Aug-19.xlsx')

324 1579 1217


C:\Users\annautiyal\AppData\Local\Continuum\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:912: UserWarning: Ignoring URL 'https://nww.finsys.sbs.nhs.uk:4465/oa_html/rf.jsp?function_id=28892&resp_id=-1&resp_appl_id=-1&security_group_id=0&lang_code=us&params=euic2seusd3dye8d.vr9d.qxtmlxxsm3pevercr9vplvvjc2fgklm-d1rpw.n0bc4-qr9ohq71582wy6efnqrdzrvdjtswargsl1bu1gmpuojqjpicciusqoonog7nvq8.uvkcird6magjz3xpulkq&oas=qqsk8dtbrnzftduexhnw8g..' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
